In [2]:
%load_ext autoreload
%autoreload

import app

In [3]:
URL_LIST = [
    "https://www.reddit.com/r/Unity3D.rss",
    "https://www.reddit.com/r/unity.rss",
    "https://www.reddit.com/r/playmygame.rss",
    "https://www.reddit.com/r/indiegames.rss",
    "https://www.reddit.com/r/gamedev.rss",
]
entries = []
for url in URL_LIST:
    res = app.fetch_entries(url)
    entries.extend(res)

In [17]:
consumed_capacity = 0
for entry in entries:
    try:
        result = app.table.update_item(
            Key={"entry_url": entry.link},
            UpdateExpression="SET author = :author, title = :title, updated = :updated, published = :published, content = :content",
            ExpressionAttributeValues={
                ':author': entry.author,
                ':title': entry.title,
                ':updated': entry.updated,
                ':published': entry.published,
                ':content': entry.content,
            },
            ReturnConsumedCapacity='TOTAL',
            ConditionExpression="attribute_not_exists(entry_url)"
        )
        consumed_capacity += result['ConsumedCapacity']['CapacityUnits']
        print("OK", entry.link)
    except Exception as e:
        print("NG", entry.link, e)
print("DONE", consumed_capacity)

NG https://www.reddit.com/r/Unity3D/comments/193o1pe/unity_is_cutting_ironsource/ An error occurred (ConditionalCheckFailedException) when calling the UpdateItem operation: The conditional request failed
NG https://www.reddit.com/r/Unity3D/comments/1am0315/why_not_its_not_joke_its_error_in_custom/ An error occurred (ConditionalCheckFailedException) when calling the UpdateItem operation: The conditional request failed
NG https://www.reddit.com/r/Unity3D/comments/1am1nnt/i_added_a_minigame_for_the_forge_in_my_game_what/ An error occurred (ConditionalCheckFailedException) when calling the UpdateItem operation: The conditional request failed
NG https://www.reddit.com/r/Unity3D/comments/1am4ivr/what_do_you_think_of_my_car_mechanics/ An error occurred (ConditionalCheckFailedException) when calling the UpdateItem operation: The conditional request failed
NG https://www.reddit.com/r/Unity3D/comments/1alsm85/i_compiled_a_free_app_via_unity_for_texturing/ An error occurred (ConditionalCheckFaile

0

ConditionalCheckFailedException: An error occurred (ConditionalCheckFailedException) when calling the UpdateItem operation: The conditional request failed